In [1]:
import pandas as pd

In [2]:
newzy = pd.read_csv('../FinalProject/newzy.csv', sep='|', index_col='doc_id')

/Users/matthewthomas/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [37]:
OHCO = ['doc_id', 'sent_num', 'token_num']

In [10]:
newzy.groupby('doc_source').count()

,doc_title,doc_content,doc_date,doc_url
doc_source,,,,
Breitbart,10012,10011,10012,10012
CNN,38011,36184,38011,38011
Daily Kos,12310,12307,12310,12310
Drudge Report,77063,77063,77063,77063
Fox,83415,58077,83417,83417
Google News,64079,64079,64079,64078
Guardian,12392,12392,12392,12392
NPR,30755,24708,30755,30755
New York Times,21239,21145,21240,21240


## Organize and Tokenize PowerLine Articles

In [3]:
powerline = newzy[newzy.doc_source.isin(['PowerLine'])]

In [4]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)

In [5]:
powerline.doc_content.str.len().describe()

count     9730.000000
mean      2844.561871
std       1942.034802
min         91.000000
25%       1532.250000
50%       2506.000000
75%       3692.000000
max      40421.000000
Name: doc_content, dtype: float64

In [6]:
#filter out very short articles
powerline = powerline[powerline['doc_content'].str.len() > 300]

In [7]:
len(powerline)

9664

In [8]:
#inspect articles to see what needs to be filtered
powerline.doc_content[:20]

doc_id
48                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [9]:
#get rid of the jquery stuff
powerline['doc_content'] = powerline['doc_content'].str.replace(r'if\(typeof\(jQuery.*', '')

In [10]:
powerline.doc_content[:1]

doc_id
48    (Steven Hayward) I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine.  So here’s a quick catch up guide. Old timers of Paso Robles wines all know the Dusi name.  The Dusi family’s Zinfandel vineyards go back nearly a century.  For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year.  The Dusi family seldom made their own wine—until now.  And here’s the four-minute storyboard for it.  (Footnote: I’ve met Janell Dusi a couple of times, and always said, “When are you guys going to start keeping your own grapes for yourselves?”  The time has come.) 
Name: doc_content, dtype: object

In [14]:
#Take out authors in parantheses. This will delete some other words unfortunately
powerline['doc_content'] = powerline['doc_content'].str.replace(r'\([\w\s]*', '')
powerline.doc_content[:1]


doc_id
48    ) I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine.  So here’s a quick catch up guide. Old timers of Paso Robles wines all know the Dusi name.  The Dusi family’s Zinfandel vineyards go back nearly a century.  For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year.  The Dusi family seldom made their own wine—until now.  And here’s the four-minute storyboard for it.  : I’ve met Janell Dusi a couple of times, and always said, “When are you guys going to start keeping your own grapes for yourselves?”  The time has come.) 
Name: doc_content, dtype: object

In [39]:
#Split up sentences
dfs = powerline['doc_content'].str.split(r'[.?!;:"\(\)]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sentence'})

In [40]:
dfs = dfs[~dfs['sentence'].str.match(r'^\s*$')] # Remove empty sentences
dfs.head()

sentence
doc_id                                                                                                                                       
48     1   I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine
       2                                                                                                     So here’s a quick catch up guide
       3                                                                               Old timers of Paso Robles wines all know the Dusi name
       4                                                                       The Dusi family’s Zinfandel vineyards go back nearly a century
       5    For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year

In [41]:
dfs.index.names = OHCO[:2]
dfs.head()

sentence
doc_id sent_num                                                                                                                                     
48     1          I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine
       2                                                                                                            So here’s a quick catch up guide
       3                                                                                      Old timers of Paso Robles wines all know the Dusi name
       4                                                                              The Dusi family’s Zinfandel vineyards go back nearly a century
       5           For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year

In [42]:
#split into tokens
df = dfs['sentence'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame().rename(columns={0:'token'})

In [45]:
df.index.names = OHCO
df = df[~df['token'].str.match(r'^\s*$')]
df.head()

token
doc_id sent_num token_num         
48     1        1             I’ve
                2             been
                3          falling
                4           behind
                5               on